In [24]:
#Importing libraries
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import warnings
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [2]:
#Reading the parquet file
df1=pd.read_parquet(r'fhv_tripdata_2021-01.parquet', engine='pyarrow')
#df2=pd.read_parquet(r'C:\Users\Square\Downloads\mlops_assignment\DATA\fhv_tripdata_2021-02.parquet', engine='pyarrow')

In [3]:
#Reading the nos of record in january2021
df1.shape

(1154112, 7)

In [4]:
#Datatypes of variable
df1.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
dtype: object

In [5]:
#Finding Duration time
df1['Duration']=(df1['dropOff_datetime']-df1.pickup_datetime)/pd.Timedelta(minutes=1)

In [6]:
df1.head()


dispatching_base_num     pickup_datetime    dropOff_datetime  PUlocationID  \
0               B00009 2021-01-01 00:27:00 2021-01-01 00:44:00           NaN   
1               B00009 2021-01-01 00:50:00 2021-01-01 01:07:00           NaN   
2               B00013 2021-01-01 00:01:00 2021-01-01 01:51:00           NaN   
3               B00037 2021-01-01 00:13:09 2021-01-01 00:21:26           NaN   
4               B00037 2021-01-01 00:38:31 2021-01-01 00:53:44           NaN   

   DOlocationID SR_Flag Affiliated_base_number    Duration  
0           NaN    None                 B00009   17.000000  
1           NaN    None                 B00009   17.000000  
2           NaN    None                 B00013  110.000000  
3          72.0    None                 B00037    8.283333  
4          61.0    None                 B00037   15.216667

In [7]:
#Finding avg duration time in jan
df1.Duration.mean()

19.1672240937939

In [8]:
df1=df1[(df1.Duration>=1) & (df1.Duration<=60)]

In [9]:
#Finding  % of null values for pickup location 
df1.PUlocationID.isnull().sum()/df1.shape[0]

0.8352732770722617

In [10]:
#REplacing null values with -1
df1['PUlocationID']=df1.PUlocationID.fillna('-1')
df1['DOlocationID']=df1.DOlocationID.fillna('-1')

In [21]:
train_dataset=df1[['PUlocationID','DOlocationID']].astype('str')

In [22]:
train_dict=train_dataset.to_dict(orient='records')
dv=DictVectorizer()
X=dv.fit_transform(train_dict)
Y=df1.Duration
X.shape,Y.shape

((1109826, 525), (1109826,))

In [26]:
#Splitting Data in train test split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=2)

### Model Trainning

In [27]:
lr=LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [30]:
#Prediction on train Data
pred_train=lr.predict(X_train)

In [33]:
#Evaluating model on train data
mean_squared_error(y_train,pred_train)**0.5


10.532446650347689

In [34]:
#Model evaluation on test data
mean_squared_error(y_test,lr.predict(X_test))**0.5


10.521271716685215